In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder,StandardScaler
import numpy as np
from imblearn.over_sampling import ADASYN
from imblearn.under_sampling import NearMiss



In [58]:
df = pd.read_csv(r"C:\Users\vishnu\fraud_detection.csv")

In [59]:
df.shape

(307511, 33)

In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Data columns (total 33 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Unnamed: 0                   307511 non-null  int64  
 1   SK_ID_CURR                   307511 non-null  int64  
 2   TARGET                       307511 non-null  int64  
 3   NAME_CONTRACT_TYPE           307511 non-null  object 
 4   CODE_GENDER                  307511 non-null  object 
 5   FLAG_OWN_CAR                 307511 non-null  object 
 6   FLAG_OWN_REALTY              307511 non-null  object 
 7   CNT_CHILDREN                 307511 non-null  int64  
 8   AMT_INCOME_TOTAL             307511 non-null  float64
 9   AMT_CREDIT                   307511 non-null  float64
 10  AMT_ANNUITY                  307499 non-null  float64
 11  AMT_GOODS_PRICE              307233 non-null  float64
 12  NAME_TYPE_SUITE              306219 non-null  object 
 13 

In [61]:
df.drop(columns = ['SK_ID_CURR','AMT_GOODS_PRICE','CODE_GENDER'],inplace = True)

# creating a pipeline function to move all strings to uppercase

In [62]:
def uppercase_str(df):
    object_cols = df.select_dtypes(include=['object']).columns
    df[object_cols] = df[object_cols].apply(lambda x: x.str.upper())
    return df

In [63]:
df = uppercase_str(df)

In [64]:
df.describe(include = 'all')

,Unnamed: 0,TARGET,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,NAME_TYPE_SUITE,...,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY
count,307511.000000,307511.000000,307511,307511,307511,307511.000000,3.075110e+05,3.075110e+05,307499.000000,306219,...,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,211120,307509.000000,307511.000000,307511.000000
unique,NaN,NaN,2,2,2,NaN,NaN,NaN,NaN,7,...,NaN,NaN,NaN,NaN,NaN,NaN,18,NaN,NaN,NaN
top,NaN,NaN,CASH LOANS,N,Y,NaN,NaN,NaN,NaN,UNACCOMPANIED,...,NaN,NaN,NaN,NaN,NaN,NaN,LABORERS,NaN,NaN,NaN
freq,NaN,NaN,278232,202924,213312,NaN,NaN,NaN,NaN,248526,...,NaN,NaN,NaN,NaN,NaN,NaN,55186,NaN,NaN,NaN
mean,153755.000000,0.080729,NaN,NaN,NaN,0.417052,1.687979e+05,5.990260e+05,27108.573909,NaN,...,0.999997,0.819889,0.199368,0.998133,0.281066,0.056720,NaN,2.152665,2.052463,2.031521
std,88770.923652,0.272419,NaN,NaN,NaN,0.722121,2.371231e+05,4.024908e+05,14493.737315,NaN,...,0.001803,0.384280,0.399526,0.043164,0.449521,0.231307,NaN,0.910682,0.509034,0.502737
min,0.000000,0.000000,NaN,NaN,NaN,0.000000,2.565000e+04,4.500000e+04,1615.500000,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,1.000000,1.000000,1.000000
25%,76877.500000,0.000000,NaN,NaN,NaN,0.000000,1.125000e+05,2.700000e+05,16524.000000,NaN,...,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,NaN,2.000000,2.000000,2.000000
50%,153755.000000,0.000000,NaN,NaN,NaN,0.000000,1.471500e+05,5.135310e+05,24903.000000,NaN,...,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,NaN,2.000000,2.000000,2.000000
75%,230632.500000,0.000000,NaN,NaN,NaN,1.000000,2.025000e+05,8.086500e+05,34596.000000,NaN,...,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,NaN,3.000000,2.000000,2.000000


In [65]:
if (df['AMT_ANNUITY'].dtypes == 'int64') | (df['AMT_ANNUITY'].dtypes == 'float64'):
    print('x')

x


In [66]:
def drop_high_nulls_col(df):
    # Grouped median calculation for filling missing values
    
    for cols in df.columns:
        if (df[cols].notnull().sum() / df.shape[0]) < 0.4:
            df.drop(columns=[cols], inplace=True)
            
    return df

In [67]:
df = drop_high_nulls_col(df)

In [68]:
X_col = df.drop(columns = 'TARGET').columns
y_col = ['TARGET']
group_cols = ['NAME_CONTRACT_TYPE', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']


In [69]:
from sklearn.model_selection import train_test_split,cross_val_score,KFold,RandomizedSearchCV

In [88]:
X_train,X_test,y_train,y_test = train_test_split(df[X_col],df[y_col],stratify=df[group_cols],train_size=0.8,test_size = 0.2,random_state=37)

In [89]:
x_test


,Unnamed: 0,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,NAME_TYPE_SUITE,NAME_INCOME_TYPE,...,NAME_FAMILY_STATUS_SINGLE / NOT MARRIED,NAME_FAMILY_STATUS_UNKNOWN,NAME_FAMILY_STATUS_WIDOW,NAME_HOUSING_TYPE_CO-OP APARTMENT,NAME_HOUSING_TYPE_HOUSE / APARTMENT,NAME_HOUSING_TYPE_MUNICIPAL APARTMENT,NAME_HOUSING_TYPE_OFFICE APARTMENT,NAME_HOUSING_TYPE_RENTED APARTMENT,NAME_HOUSING_TYPE_WITH PARENTS,OCCUPATION_TYPE_LABORERS
25474,25474,CASH LOANS,N,Y,1,202500.0,312840.0,29385.0,UNACCOMPANIED,WORKING,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
164685,164685,CASH LOANS,N,Y,0,54000.0,295668.0,29385.0,UNACCOMPANIED,PENSIONER,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
57398,57398,CASH LOANS,N,N,0,135000.0,283500.0,28116.0,UNACCOMPANIED,WORKING,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
134640,134640,CASH LOANS,Y,Y,1,225000.0,675000.0,29385.0,UNACCOMPANIED,COMMERCIAL ASSOCIATE,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
124450,124450,CASH LOANS,N,N,0,180000.0,142632.0,28116.0,UNACCOMPANIED,WORKING,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7793,7793,CASH LOANS,Y,Y,1,121500.0,675000.0,29385.0,UNACCOMPANIED,COMMERCIAL ASSOCIATE,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
54818,54818,CASH LOANS,Y,Y,0,135000.0,539100.0,29385.0,UNACCOMPANIED,STATE SERVANT,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
49680,49680,REVOLVING LOANS,Y,Y,2,337500.0,270000.0,13500.0,UNACCOMPANIED,WORKING,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
128705,128705,CASH LOANS,Y,Y,0,90000.0,521280.0,29385.0,UNACCOMPANIED,WORKING,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [71]:
def null_dict(df,grouped_cols):
    imputation_dict = {}
    # Calculate the sum of null values for each column
    null_sum = df.isnull().sum()

    # Filter columns where the sum of null values is greater than 0
    columns_with_nulls = null_sum[null_sum > 0].index

    # Select only the columns from the original DataFrame that have more than 0 null values
    null_cols = df.loc[:, columns_with_nulls].columns
    for col in null_cols:
        if (df[col].dtypes == 'int64') | (df[col].dtypes == 'float64'):
            imputation_dict.update({col : df.groupby(grouped_cols)[col].median().to_dict()})
        elif (df[col].dtypes == 'O'):
            imputation_dict.update({col : df.groupby(grouped_cols)[col].agg(pd.Series.mode).to_dict()})
                
    return imputation_dict

In [72]:
imput_dict = null_dict(X_train,group_cols)

In [73]:
def fillnulls(data,grouped_cols,imput_dict):
    imputation_dict = imput_dict
    
    col_x,col_y,col_z =grouped_cols
    
    for col in imputation_dict:
        for val in imputation_dict[col]:
            val_x,val_y,val_z = val
            data.loc[(data[col_x] == val_x) & (data[col_z] == val_z) & (data[col_z] == val_z),col] = imputation_dict[col][val]
    return data

In [74]:
X_train = fillnulls(X_train,group_cols,imput_dict)
X_test = fillnulls(X_test,group_cols,imput_dict)

# pipeline function for one hot encoding and converting back

In [75]:
def oneh(df,X_train,X_test):
    object_cols = df.select_dtypes(include=['object']).columns
    oneh = OneHotEncoder(sparse_output=False)
    oneh.fit(X_train[object_cols])
    col_names = oneh.get_feature_names_out()
    X_train_dummies = pd.DataFrame(oneh.transform(X_train[object_cols]),columns = col_names)
    X_train_dummies.index = X_train.index
    X_train = pd.merge(X_train,X_train_dummies,left_index=True,right_index=True)

    X_test_dummies = pd.DataFrame(oneh.transform(X_test[object_cols]),columns = col_names)
    X_test_dummies.index = X_test.index
    X_test = pd.merge(X_test,X_test_dummies,left_index=True,right_index=True)
    return X_train,X_test

In [76]:
x,y = oneh(df,X_train,X_test)
x.sample(3)

,Unnamed: 0,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,NAME_TYPE_SUITE,NAME_INCOME_TYPE,...,NAME_FAMILY_STATUS_SINGLE / NOT MARRIED,NAME_FAMILY_STATUS_UNKNOWN,NAME_FAMILY_STATUS_WIDOW,NAME_HOUSING_TYPE_CO-OP APARTMENT,NAME_HOUSING_TYPE_HOUSE / APARTMENT,NAME_HOUSING_TYPE_MUNICIPAL APARTMENT,NAME_HOUSING_TYPE_OFFICE APARTMENT,NAME_HOUSING_TYPE_RENTED APARTMENT,NAME_HOUSING_TYPE_WITH PARENTS,OCCUPATION_TYPE_LABORERS
19338,19338,CASH LOANS,N,Y,0,112500.0,675000.0,29385.0,UNACCOMPANIED,PENSIONER,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
230964,230964,CASH LOANS,N,Y,0,90000.0,318528.0,29385.0,UNACCOMPANIED,PENSIONER,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
288370,288370,CASH LOANS,N,Y,0,90000.0,148500.0,29385.0,UNACCOMPANIED,WORKING,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [77]:
y.sample(2)

,Unnamed: 0,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,NAME_TYPE_SUITE,NAME_INCOME_TYPE,...,NAME_FAMILY_STATUS_SINGLE / NOT MARRIED,NAME_FAMILY_STATUS_UNKNOWN,NAME_FAMILY_STATUS_WIDOW,NAME_HOUSING_TYPE_CO-OP APARTMENT,NAME_HOUSING_TYPE_HOUSE / APARTMENT,NAME_HOUSING_TYPE_MUNICIPAL APARTMENT,NAME_HOUSING_TYPE_OFFICE APARTMENT,NAME_HOUSING_TYPE_RENTED APARTMENT,NAME_HOUSING_TYPE_WITH PARENTS,OCCUPATION_TYPE_LABORERS
32367,32367,CASH LOANS,N,Y,0,112500.0,781920.0,29385.0,UNACCOMPANIED,WORKING,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
275681,275681,CASH LOANS,Y,Y,0,180000.0,219042.0,29385.0,UNACCOMPANIED,WORKING,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [78]:
X_train = x
x_test = y

In [79]:
X_train.columns

Index(['Unnamed: 0', 'NAME_CONTRACT_TYPE', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY',
       'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY',
       'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE',
       'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'REGION_POPULATION_RELATIVE',
       'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH',
       'FLAG_MOBIL', 'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE', 'FLAG_CONT_MOBILE',
       'FLAG_PHONE', 'FLAG_EMAIL', 'OCCUPATION_TYPE', 'CNT_FAM_MEMBERS',
       'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY',
       'NAME_CONTRACT_TYPE_CASH LOANS', 'NAME_CONTRACT_TYPE_REVOLVING LOANS',
       'FLAG_OWN_CAR_N', 'FLAG_OWN_CAR_Y', 'FLAG_OWN_REALTY_N',
       'FLAG_OWN_REALTY_Y', 'NAME_TYPE_SUITE_UNACCOMPANIED',
       'NAME_INCOME_TYPE_BUSINESSMAN', 'NAME_INCOME_TYPE_COMMERCIAL ASSOCIATE',
       'NAME_INCOME_TYPE_MATERNITY LEAVE', 'NAME_INCOME_TYPE_PENSIONER',
       'NAME_INCOME_TYPE_STATE SERVANT', 'NAME_INCOME_

In [90]:
x_test

,Unnamed: 0,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,NAME_TYPE_SUITE,NAME_INCOME_TYPE,...,NAME_FAMILY_STATUS_SINGLE / NOT MARRIED,NAME_FAMILY_STATUS_UNKNOWN,NAME_FAMILY_STATUS_WIDOW,NAME_HOUSING_TYPE_CO-OP APARTMENT,NAME_HOUSING_TYPE_HOUSE / APARTMENT,NAME_HOUSING_TYPE_MUNICIPAL APARTMENT,NAME_HOUSING_TYPE_OFFICE APARTMENT,NAME_HOUSING_TYPE_RENTED APARTMENT,NAME_HOUSING_TYPE_WITH PARENTS,OCCUPATION_TYPE_LABORERS
25474,25474,CASH LOANS,N,Y,1,202500.0,312840.0,29385.0,UNACCOMPANIED,WORKING,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
164685,164685,CASH LOANS,N,Y,0,54000.0,295668.0,29385.0,UNACCOMPANIED,PENSIONER,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
57398,57398,CASH LOANS,N,N,0,135000.0,283500.0,28116.0,UNACCOMPANIED,WORKING,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
134640,134640,CASH LOANS,Y,Y,1,225000.0,675000.0,29385.0,UNACCOMPANIED,COMMERCIAL ASSOCIATE,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
124450,124450,CASH LOANS,N,N,0,180000.0,142632.0,28116.0,UNACCOMPANIED,WORKING,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7793,7793,CASH LOANS,Y,Y,1,121500.0,675000.0,29385.0,UNACCOMPANIED,COMMERCIAL ASSOCIATE,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
54818,54818,CASH LOANS,Y,Y,0,135000.0,539100.0,29385.0,UNACCOMPANIED,STATE SERVANT,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
49680,49680,REVOLVING LOANS,Y,Y,2,337500.0,270000.0,13500.0,UNACCOMPANIED,WORKING,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
128705,128705,CASH LOANS,Y,Y,0,90000.0,521280.0,29385.0,UNACCOMPANIED,WORKING,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [91]:
object_cols = df.select_dtypes(include=['object']).columns
X_train.drop(columns = object_cols,inplace = True)
X_test.drop(columns = object_cols,inplace = True)

In [92]:
X_test.columns

Index(['Unnamed: 0', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT',
       'AMT_ANNUITY', 'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH',
       'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'FLAG_MOBIL',
       'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE', 'FLAG_CONT_MOBILE', 'FLAG_PHONE',
       'FLAG_EMAIL', 'CNT_FAM_MEMBERS', 'REGION_RATING_CLIENT',
       'REGION_RATING_CLIENT_W_CITY'],
      dtype='object')

# creating a pipeline function for midsampling


In [82]:
def mid_sampling(X_train,Y_train,target_minority_samples=180000):
    
    def oversample_with_adasyn(X, y, target_minority_samples=180000):
        # Identify minority class
        classes, counts = np.unique(y, return_counts=True)
        minority_class = classes[np.argmin(counts)]
        current_minority_samples = counts[np.argmin(counts)]

    # Calculate how many new samples we need to generate for the minority class
        if current_minority_samples >= target_minority_samples:
            raise ValueError("The minority class already has more or equal samples than the target_minority_samples.")
    
        n_samples_to_generate = target_minority_samples - current_minority_samples

    # Set the sampling strategy for ADASYN
    # This will tell ADASYN to generate the specified number of samples for the minority class
        sampling_strategy = {minority_class: n_samples_to_generate}

    # Create the ADASYN instance and fit it to the data
        adasyn = ADASYN(sampling_strategy=sampling_strategy, n_neighbors=5, random_state=42)
        X_resampled, y_resampled = adasyn.fit_resample(X, y)

        return X_resampled, y_resampled
   
    def undersample_with_nearmiss(X, y):
    
    # Identify majority and minority classes
        classes, counts = np.unique(y, return_counts=True)
        minority_class_count = min(counts)
    
    # Set the sampling strategy for NearMiss
    # This will balance the majority class with the minority class
        nm = NearMiss()
    
    # Perform the undersampling
        X_resampled, y_resampled = nm.fit_resample(X, y)
    
    # Check if undersampling was successful
        if len(np.unique(y_resampled, return_counts=True)[1]) != 2 or np.unique(y_resampled, return_counts=True)[1][0] != minority_class_count:
            raise ValueError("NearMiss did not undersample correctly.")
    
        return X_resampled, y_resampled

    X_resampled,y_resampled = oversample_with_adasyn(X_train,y_train,target_minority_samples)
    X_train,Y_train = undersample_with_nearmiss(X_resampled,y_resampled)
    
    return X_train,Y_train

In [83]:
x, y = mid_sampling(X_train,Y_train)

In [84]:
X_train = x
Y_train = y

In [85]:
Y_train.groupby(['TARGET']).size()

TARGET
0    161767
1    161767
dtype: int64

# create a pipeline for scaling

In [99]:
def scaling(X_train,X_test):
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train_scaled = scaler.transform(X_train)
    X_train = pd.DataFrame(X_train_scaled, index=X_train.index, columns=X_train.columns)
    X_test_scaled = scaler.transform(X_test)
    X_test = pd.DataFrame(X_test_scaled, index=X_test.index, columns=X_test.columns)
    return X_train,X_test

In [100]:
x, y =scaling(X_train,X_test)


In [103]:
X_train = x
X_test = y

In [104]:
X_train

,Unnamed: 0,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY
33452,-1.352442,-0.578010,0.042709,0.066444,-0.310934,-0.462396,-0.037623,-0.471456,-0.913316,1.665357,0.002016,0.468718,-0.499412,0.043188,-0.625400,-0.245489,-1.264507,-0.102640,-0.062757
296793,1.614668,-0.578010,0.301952,-1.107650,-0.826781,0.127464,-0.210143,-0.511653,-1.698146,1.744888,0.002016,0.468718,-0.499412,0.043188,-0.625400,-0.245489,-1.264507,-0.102640,-0.062757
278979,1.413954,-0.578010,0.561195,0.748136,0.429977,0.563878,-0.692647,-0.456472,-1.352140,0.249711,0.002016,0.468718,-0.499412,0.043188,1.598976,-0.245489,-0.166286,-0.102640,-0.062757
2459,-1.701645,-0.578010,-0.043705,-0.041121,-0.574127,0.563878,-0.549453,-0.462672,0.698352,0.682490,0.002016,0.468718,-0.499412,0.043188,-0.625400,-0.245489,-0.166286,-0.102640,-0.062757
128604,-0.280347,-0.578010,0.734023,2.157925,1.983100,1.832862,-0.238781,-0.470380,1.070755,1.579862,0.002016,0.468718,2.002356,0.043188,1.598976,-0.245489,-1.264507,-2.067939,-2.051264
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117222,-0.408590,2.195662,-0.302948,-0.633309,-0.965973,-0.993205,0.048063,-0.457526,1.309752,-0.896194,0.002016,0.468718,-0.499412,0.043188,-0.625400,-0.245489,2.030155,-0.102640,-0.062757
183364,0.336643,-0.578010,0.042709,1.057955,1.961709,1.079147,0.506511,-0.454426,-0.765150,-0.655614,0.002016,0.468718,2.002356,0.043188,-0.625400,-0.245489,-0.166286,-0.102640,-0.062757
2463,-1.701600,-0.578010,-0.302948,-0.745792,-0.368594,-0.044485,0.285420,-0.458758,-0.408074,-1.506592,0.002016,0.468718,-0.499412,0.043188,-0.625400,-0.245489,-0.166286,1.862660,1.925750
250199,1.089685,-0.578010,-0.302948,0.412834,-0.117181,-0.407682,-0.977888,2.133502,-0.577245,-0.505168,0.002016,-2.133477,-0.499412,0.043188,-0.625400,-0.245489,-0.166286,-0.102640,-0.062757


In [105]:
X_test

,Unnamed: 0,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY
25474,-1.442331,0.808826,0.129124,-0.710205,-0.418505,1.832862,-0.116895,-0.478400,0.072191,1.935099,0.002016,0.468718,-0.499412,0.043188,-0.625400,-0.245489,-0.166286,-2.067939,-2.051264
164685,0.126184,-0.578010,-0.441210,-0.752855,-1.127176,-0.715224,-1.651927,2.133502,0.035859,-1.238176,0.002016,-2.133477,-0.499412,0.043188,-0.625400,-0.245489,-1.264507,-0.102640,-0.062757
57398,-1.082638,-0.578010,-0.130119,-0.783077,-0.315894,-0.407682,-0.270398,-0.455566,0.011164,1.475809,0.002016,0.468718,-0.499412,0.043188,-0.625400,-0.245489,-1.264507,-0.102640,-0.062757
134640,-0.212339,0.808826,0.215538,0.189299,0.378206,-0.811788,-0.370519,-0.456712,-0.431917,1.204742,0.002016,0.468718,-0.499412,0.043188,1.598976,-0.245489,0.931934,-0.102640,-0.062757
124450,-0.327151,-0.578010,0.042709,-1.132954,-1.091525,-1.282173,1.319161,-0.460669,-0.022045,0.033652,0.002016,0.468718,-0.499412,0.043188,-0.625400,-0.245489,-0.166286,1.862660,1.925750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7793,-1.641546,0.808826,-0.181968,0.189299,-0.508097,0.270936,0.566996,-0.475788,-0.755783,0.634772,0.002016,0.468718,2.002356,0.043188,1.598976,-0.245489,0.931934,-0.102640,-0.062757
54818,-1.111707,-0.578010,-0.130119,-0.148239,-0.072230,-0.774420,-0.189065,-0.505636,1.383552,1.721029,0.002016,0.468718,2.002356,0.043188,1.598976,-0.245489,-0.166286,-0.102640,-0.062757
49680,-1.169598,2.195662,0.647609,-0.816608,-0.937763,-0.191715,-0.017462,-0.468703,-0.714058,-0.629104,0.002016,0.468718,-0.499412,0.043188,-0.625400,-0.245489,2.030155,1.862660,1.925750
128705,-0.279209,-0.578010,-0.302948,-0.192498,-0.286133,0.127464,-0.850732,-0.484635,-0.566743,-0.062447,0.002016,0.468718,2.002356,0.043188,1.598976,-0.245489,-0.166286,-0.102640,-0.062757
